In [1]:
import numpy as np 
import pandas as pd
from scipy.stats import skew
from scipy.signal import find_peaks
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, roc_auc_score
from itertools import chain
from collections import Counter
import itertools
import pickle
import os

##**Functions**

In [2]:
def extract_list_feats(list_name: str, data, features_name: list, base=None):
    """
    Extract Features from vector.
    :param list_name: Vector to extract features from.
    :param data: Dataset to extract features from.
    :param features_name: Feature list to add new feature names to.
    :param base: Disable the use of features.
    :return: Data with features, updated feature name list.
    """

    if base is None:
        base = DEFAULT_TRUE_LIST

    data[f'max_{list_name}'] = data[list_name].apply(np.max)
    if base[0]:
        features_name += [f'max_{list_name}']

    data[f'min_{list_name}'] = data[list_name].apply(np.min)
    if base[1]:
        features_name += [f'min_{list_name}']

    data[f'mean_{list_name}'] = data[list_name].apply(np.mean)
    if base[2]:
        features_name += [f'mean_{list_name}']

    data[f'median_{list_name}'] = data[list_name].apply(np.median)
    if base[3]:
        features_name += [f'median_{list_name}']

    data[f'std_{list_name}'] = data[list_name].apply(np.std)
    if base[4]:
        features_name += [f'std_{list_name}']

    data[f'skew_{list_name}'] = data[list_name].apply(skew)
    if base[5]:
        features_name += [f'skew_{list_name}']

    data[f'max_sub_min_{list_name}'] = data[list_name].apply(lambda x: np.max(x) - np.min(x))
    if base[6]:
        features_name += [f'max_sub_min_{list_name}']

    return data, features_name

In [3]:
def extract_features(data, bases=None):
    """
    Extract features from data.
    :param data: Dataset of time windows.
    :param bases: Dictionary with values of bool lists of size 7 and keys of the names of the vectors to extract
    features from
    :return: new dataset with extracted features, training feature name list
    """

    if bases is None:
        bases = DEFAULT_TRUE_DICT

    features_name = []
    data['RSSI_diffs'] = data.RSSI.apply(lambda x: x[1:] - x[:-1])
    data['RSSI_diffs_abs'] = data.RSSI.apply(lambda x: abs(x[1:] - x[:-1]))
    data['RSSI_median_dist'] = data.RSSI.apply(lambda x: abs(x - np.median(x)))

    data, features_name = extract_list_feats('RSSI', data, features_name, base=bases['RSSI'])
    data, features_name = extract_list_feats('RSSI_diffs', data, features_name, base=bases['RSSI_diffs'])
    data, features_name = extract_list_feats('RSSI_diffs_abs', data, features_name, base=bases['RSSI_diffs_abs'])
    data, features_name = extract_list_feats('RSSI_median_dist', data, features_name, base=bases['RSSI_median_dist'])

    data['max_count_same_value_RSSI'] = data.RSSI.apply(lambda x: np.max(np.unique(x, return_counts=True)[1]))
    features_name += ['max_count_same_value_RSSI']

    data['RSSI_peaks'] = data.RSSI.apply(lambda x: len(find_peaks(x)[0]))
    features_name += ['RSSI_peaks']

    data['RSSI_diffs_peaks'] = data.RSSI_diffs.apply(lambda x: len(find_peaks(x)[0]))
    features_name += ['RSSI_diffs_peaks']

    data['peak_ratio_diffs_RSSI'] = data.apply(
        lambda x: x['RSSI_diffs_peaks'] / x['RSSI_peaks'] if x['RSSI_peaks'] > 0 else 0, axis=1)
    features_name += ['peak_ratio_diffs_RSSI']

    data['RSSI_values_count'] = data.RSSI.apply(lambda x: len(np.unique(x)))
    features_name += ['RSSI_values_count']

    return data, features_name

In [4]:
def window(full_signal: np.ndarray, size: int = 360, stride: int = 360):
    """
    Take a long vector of signals and creates time windows of size "size" and stride of size "stride"
    :param full_signal: the signal to make time windows from
    :param size: size of each time window
    :param stride: time window stride (step size). When window size <= stride it's mean that there is not overlap between the windows.
    :return: time windows of the signal
    """
    return np.lib.stride_tricks.sliding_window_view(full_signal, size)[0::stride]

In [5]:
def make_data(X, y, window_size: int = 360, stride: int = 360):
    """
    Make data for training a model: making windows, adding metadata information to the time windows dataframe, removing
    windows with change in Num_People
    :param X: the data.
    :param y: the labels
    :param window_size: size of each time window
    :param stride: time window stride (step size). When window size <= stride it's mean that there is not overlap between the windows.
    :return: windowed RSSI DataFrame , labels dataframe
    """
    
    X['Num_People'] = y
    multi_vals = X.groupby(['Device_ID']).apply(lambda x: x.nunique() == 1).all()
    single_vals = list(multi_vals[multi_vals].index)
    multi_vals = list(multi_vals[~multi_vals].index)
    windows_df = X.groupby(['Device_ID']).RSSI.apply(
        lambda x: window(x.values, window_size, stride)).explode().to_frame().reset_index()
    for col in (multi_vals + single_vals):
        windows_df[col] = X.groupby(['Device_ID'])[col].apply(
            lambda x: window(x.values, window_size, stride)).explode().reset_index(drop=True).values
    for col in single_vals:
        windows_df[col] = windows_df[col].apply(lambda x: x[0])
    
    df = windows_df
    df['change'] = df.Num_People.apply(lambda x: (len(np.unique(x)) > 1))
    dfx = df[~df['change']]
    df = dfx.copy()
    df.Num_People = df.Num_People.apply(lambda x: x[0])
    df.drop(columns='change', inplace=True)
    return df.drop(columns='Num_People'), df.Num_People

In [6]:
def pre_data(data, RSSI_value_selection, window_size, stride):
    """
    Full preprocessing of the data - train_x, train_y split, feature extraction,
    remove data that is smaller than the selected size window, etc.
    :param data: the row data.
    :param RSSI_value_selection: Which signal values to use.
    :param window_size: size of each time window
    :param stride: time window stride (step size). When window size <= stride it's mean that there is not overlap between the windows.
    :return: train set x (with extracted features per window), train set y
    """
    if RSSI_value_selection=="RSSI_Left":
        data["RSSI"] = data.RSSI_Left
    elif RSSI_value_selection=="RSSI_Right":
        data["RSSI"] = data.RSSI_Right
    elif RSSI_value_selection=="Min":
        data["RSSI"] = data[['RSSI_Left','RSSI_Right']].min(axis=1).values
    elif RSSI_value_selection=="Max":
        data["RSSI"] = data[['RSSI_Left','RSSI_Right']].max(axis=1).values
    else: 
        data["RSSI"] = np.ceil(data[['RSSI_Left','RSSI_Right']].mean(axis=1).values).astype('int')

    data.drop(['Room_Num'], axis=1, inplace=True)
    data.dropna(subset = ["Num_People"], inplace=True)

    for dev_id in list(set(data.Device_ID)):
        sub_dev_id = data.loc[data.Device_ID == dev_id]
        if len(sub_dev_id) < window_size:
            data = data[data.Device_ID != dev_id]
    train_x, train_y, raw_x = create_features(data, window_size, stride)
    train_x= train_x.reset_index(drop = True)
    train_y= train_y.reset_index(drop = True)
    train_x.drop('Device_ID', axis=1, inplace=True)
    return train_x, train_y, raw_x

In [7]:
def create_features(data, window_size, stride):
    """
    Feature engineering: 
    :param data: the data
    :param window_size: size of each time window
    :param stride: time window stride (step size). When window size <= stride it's mean that there is not overlap between the windows.
    :return: full dataset (with extracted features), train set y
    """

    X, y = data.drop(columns='Num_People'), data['Num_People']
    X, y = make_data(X, y, window_size=window_size, stride=stride)
    X_features, train_feat = extract_features(X.copy())
    train_feat.append('Device_ID')
    X_features = X_features[train_feat]
    return X_features, y, X

#**Model**

##**Data preparation**


Download training data

In [8]:
# !gdown -O ../data/mafat_wifi_challenge_training_set_v1.csv 'https://drive.google.com/uc?id=121CbFZbU6kAWNjmjZF232DsiGF2-BoYy'

Read traning data to dataframe

In [9]:
data = pd.read_csv('../data/mafat_wifi_challenge_training_set_v1.csv')

window_size - defines the number of timestamps in each window

window_stride - defines the shift between windows, i.e., for window_size = 360 and window_stride = 1: timestamps 0 - 359 will be selected for the first window and timestamps 1-360 will be selected for the second window. And so on for the rest of the windows for each device.

In [10]:
window_size = 360 #@param {type:"integer"}
window_stride = 360 #@param {type:"integer"}

Select the signal values to do the feature - engineering ("extract_feature" function): RSSI_Left/ RSSI_Right/ the minimum value ​​between the signals/ The maximum value ​​between the signals/ average of signals

In [11]:
RSSI_value_selection = "Average" #@param ["RSSI_Left","RSSI_Right","Min","Max","Average"]

In [12]:
"""
Lists of features to extract from each vector
"""

DEFAULT_TRUE_LIST = [True] * 7
DEFAULT_TRUE_DICT = {
    'RSSI': [True, False, False, False, True, True, True],
    'RSSI_diffs': [True, True, True, False, True, True, True],
    'RSSI_diffs_abs': [False, False, True, True, True, False, True],
    'RSSI_median_dist': [True, False, True, True, True, False, True]
}

Preprocess training data

In [13]:
data_train_x, data_train_y, raw_x = pre_data(data, RSSI_value_selection, window_size, window_stride)

##**Train RandomForestClassifier model**

###**Track 1**

####**Convert classes to 0/1**

In [14]:
# Convert classes to 0/1 to evaluate the model's score for predicting room occupancy
# in Track 1 you are required to predict probability for room occupancy (in the range of 0-1).
# however, the data is used for both tracks, and it contains the raw number of people
# in the room, here we convert the raw data to 0 or 1.

data_train_track1 = data_train_y.copy()
data_train_track1.loc[data_train_y>0] = 1

In [15]:
data_train_x['y_track1'] = data_train_track1

In [16]:
data_train_x['y_track2'] = data_train_y

In [17]:
data_train_x.to_csv("processed_rows.csv", index=False)

In [35]:
import csv
a=raw_x[:]['RSSI']#.to_csv("raw_rows.csv", index=False)
b=pd.DataFrame(a.to_list())


In [ ]:
df = pd.concat([b, data_train_x['y_track1'], data_train_x['y_track2']], axis=1)

In [37]:
df.to_csv("raw_rows.csv", index=False)

In [39]:
df.drop(['y_track1', 'y_track2'], axis=1, inplace=True)

In [40]:
df_combined = pd.concat([df, data_train_x], axis=1)

In [41]:
df_combined.to_csv("combined_rows.csv", index=False)

####**Fit Random Forest estimator to all training set**
#### No train-validation split is used (we use the submission as validation)

In [ ]:
#Train the model on all training data and calculate the AUC metric for the first track
rfc  = RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=2,
                              n_estimators=350, random_state=0, class_weight="balanced", bootstrap = True)

rfc.fit(data_train_x, data_train_track1)

train_predict_classification = rfc.predict(data_train_x)
print(f'The auc for all training set: {round(roc_auc_score(data_train_track1, rfc.predict_proba(data_train_x)[:,1], average= None),3)}')

####**Save model - track 1**

In [ ]:
# save model weights
filename = "model_track_1.sav"
pickle.dump(rfc, open(filename, 'wb'))

##**Prepare submmision**

**Attention!**

Full submission includes the following files in a zip archive:
1.   model.py (**must**) - contains a class named "model". The class must have implementations of "load", "__init__" and "predict" functions:
    *    __init__ - initialization function of the model class.
    *   load - a function that loads the model and model weights.
    *   predict - a function that receives one window each time (as a DataFrame) and returns a one value prediction.
    * **The file may contain other functions (within the class or outside of it)**
    * imports used by the class must be compatible with the permitted python packages.


2. metadata (**must**)
    * contain the command for running the model file - **do not change this file**


3.   model weights (**optional**)
    * in this example, we demonstrate how to save a Random Forest classifier   weights. however, these can be any kind of weights as long as they are compatible with the model and the permitted python packages. 
    * if the model depends on these weights, this file is mandatory.  


4. Helper_func.py (**optional**)
    * This file contains helper functions. The file can have a different name as long as it is compatible with model.py
    * if the model depends on these weights, this file is mandatory.  


Running the following cells will generate a zip file with a valid submission for track 1.

Notice the minor changes that can be made to make it a valid submission for track 2 .

This is the baseline submission, you can check it's score on the leaderboard.

In [ ]:
%%writefile helper_func.py
import numpy as np   
import pandas as pd
from scipy.stats import skew
from scipy.signal import find_peaks

def extract_features(X, bases=None):
    """
    Extract features from data.
    :param X: Dataset of time windows.
    :param bases: Dictionary with values of bool lists of size 7 and keys of the names of the vectors to extract
    features from
    :return: new dataset with extracted features, training feature name list
    """
    # Restructure dataframe to fit preprocessing features extraction functions - changing the dataframe to have one row.
    # Each column is compressed to a list.
    data = pd.DataFrame(columns=X.columns)
    for col in data.columns:
        data.loc[0,col]= np.array(X[col])

    if bases is None:
        bases = {
        'RSSI': [True, False, False, False, True, True, True],
        'RSSI_diffs': [True, True, True, False, True, True, True],
        'RSSI_diffs_abs': [False, False, True, True, True, False, True],
        'RSSI_median_dist': [True, False, True, True, True, False, True]
    } 

    features_name = []
    data['RSSI_diffs'] = data.RSSI.apply(lambda x: x[1:] - x[:-1])
    data['RSSI_diffs_abs'] = data.RSSI.apply(lambda x: abs(x[1:] - x[:-1]))
    data['RSSI_median_dist'] = data.RSSI.apply(lambda x: abs(x - np.median(x)))

    data, features_name = extract_list_feats('RSSI', data, features_name, base=bases['RSSI'])
    data, features_name = extract_list_feats('RSSI_diffs', data, features_name, base=bases['RSSI_diffs'])
    data, features_name = extract_list_feats('RSSI_diffs_abs', data, features_name, base=bases['RSSI_diffs_abs'])
    data, features_name = extract_list_feats('RSSI_median_dist', data, features_name, base=bases['RSSI_median_dist'])

    data['max_count_same_value_RSSI'] = data.RSSI.apply(lambda x: np.max(np.unique(x, return_counts=True)[1]))
    features_name += ['max_count_same_value_RSSI']

    data['RSSI_peaks'] = data.RSSI.apply(lambda x: len(find_peaks(x)[0]))
    features_name += ['RSSI_peaks']

    data['RSSI_diffs_peaks'] = data.RSSI_diffs.apply(lambda x: len(find_peaks(x)[0]))
    features_name += ['RSSI_diffs_peaks']

    data['peak_ratio_diffs_RSSI'] = data.apply(
        lambda x: x['RSSI_diffs_peaks'] / x['RSSI_peaks'] if x['RSSI_peaks'] > 0 else 0, axis=1)
    features_name += ['peak_ratio_diffs_RSSI']

    data['RSSI_values_count'] = data.RSSI.apply(lambda x: len(np.unique(x)))
    features_name += ['RSSI_values_count']

    return data, features_name

def extract_list_feats(list_name: str, data, features_name: list, base=None):
    """
    Extract Features from vector.
    :param list_name: Vector to extract features from.
    :param data: Dataset to extract features from.
    :param features_name: Feature list to add new feature names to.
    :param base: Disable the use of features.
    :return: Data with features, updated feature name list.
    """

    if base is None:
        base = DEFAULT_TRUE_LIST

    data[f'max_{list_name}'] = data[list_name].apply(np.max)
    if base[0]:
        features_name += [f'max_{list_name}']

    data[f'min_{list_name}'] = data[list_name].apply(np.min)
    if base[1]:
        features_name += [f'min_{list_name}']

    data[f'mean_{list_name}'] = data[list_name].apply(np.mean)
    if base[2]:
        features_name += [f'mean_{list_name}']

    data[f'median_{list_name}'] = data[list_name].apply(np.median)
    if base[3]:
        features_name += [f'median_{list_name}']

    data[f'std_{list_name}'] = data[list_name].apply(np.std)
    if base[4]:
        features_name += [f'std_{list_name}']

    data[f'skew_{list_name}'] = data[list_name].apply(skew)
    if base[5]:
        features_name += [f'skew_{list_name}']

    data[f'max_sub_min_{list_name}'] = data[list_name].apply(lambda x: np.max(x) - np.min(x))
    if base[6]:
        features_name += [f'max_sub_min_{list_name}']

    return data, features_name
    
def preprocess(X, RSSI_value_selection):
    """
    Calculate the features on the selected RSSI on the test set
    :param X: Dataset to extract features from.
    :param RSSI_value_selection: Which signal values to use- - in our case it is Average.
    :return: Test x dataset with features
    """
    if RSSI_value_selection=="RSSI_Left":
        X["RSSI"] = X.RSSI_Left
    elif RSSI_value_selection=="RSSI_Right":
        X["RSSI"] = X.RSSI_Right
    elif RSSI_value_selection=="Min":
        X["RSSI"] = X[['RSSI_Left','RSSI_Right']].min(axis=1).values
    elif RSSI_value_selection=="Max":
        X["RSSI"] = X[['RSSI_Left','RSSI_Right']].max(axis=1).values
    else: 
        X["RSSI"] = np.ceil(X[['RSSI_Left','RSSI_Right']].mean(axis=1).values).astype('int')

    X, features_name = extract_features(X)
    X.drop('Device_ID', axis=1, inplace=True)
    return X[features_name]

In [ ]:
%%writefile model.py

import pickle
import numpy as np
from os.path import isfile
import joblib
from sklearn.ensemble import RandomForestClassifier
from helper_func import preprocess
import os

class model:
    def __init__(self):
        '''
        Init the model
        '''

        self.model  = RandomForestClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=2,
                              n_estimators=350, random_state=0, class_weight="balanced", bootstrap = True)
        self.RSSI_value_selection = 'Average'

    def predict(self, X):
        '''
        Edit this function to fit your model.

        This function should provide predictions of labels on (test) data.
        Make sure that the predicted values are in the correct format for the scoring
        metric.
        preprocess: it our code for add feature to the data before we predict the model.
        :param X: is DataFrame with the columns - 'Time', 'Device_ID', 'Rssi_Left','Rssi_Right'. 
                  X is window of size 360 samples time, shape(360,4).
        :return: a float value of the prediction for class 1 (the room is occupied).
        '''
        # preprocessing should work on a single window, i.e a dataframe with 360 rows and 4 columns
        X = preprocess(X,self.RSSI_value_selection)
        y = self.model.predict_proba(X)[:,1][0]
        
        '''
        Track 2 - for track 2 we naively assume that the model from track-1 predicts 0/1 correctly. 
        We use that assumption in the following way:
        when the room is occupied (1,2,3 - model predicted 1) we assign the majorty class (2) as prediction.       
        '''
        #y = 0 if y<0.5 else 2
        return y

    def load(self, dir_path):
        '''
        Edit this function to fit your model.

        This function should load the model that you trained on the train set.
        :param dir_path: A path for the folder the model is submitted 
        '''
        model_name = 'model_track_1.sav' 
        model_file = os.path.join(dir_path, model_name)
        self.model = joblib.load(model_file)

In [ ]:
%%writefile metadata
command: python3 $program/model.py $input $output

zip the files to submit

In [ ]:
!zip -r submission.zip model.py helper_func.py metadata model_track_1.sav

*You can use this notebook to save your file, download it, and submit it on CodaLab.

To download the zip file, use the file manager panel.
Use View > Table of contents to show the sidebar then click the Files tab. Right-click the file and select Download.

##**Example- Prediction with the submitted model**

In this section, we demonstrate how to predict with the submitted model  on window (360 samples).

###download and read one window for prediction

(An example is based on the train set)

In [ ]:
!gdown -O one_window_for_demo.csv https://drive.google.com/uc?id=1kVAMV-zEn2bGLLtMOYA7-gVLnofCo3m_

In [ ]:
X = pd.read_csv('/content/one_window_for_demo.csv')

In [ ]:
print(X.head(10))
print(f'window shape: {len(X)}')

###Create object model, load and predict

Unzip the submission files

In [ ]:
!unzip -o '/content/submission.zip'

Create model object, load and predict

In [ ]:
from model import *
M = model()
M.load('')
Y_test=[]
unique_windows = list(set(X.Num_Window))
for window in unique_windows:
   X_test_window = X.loc[X['Num_Window'] == window]
   X_test_window.drop('Num_Window', axis=1, inplace=True)
   Y_test.append(M.predict(X_test_window))

print(f'Occupancy prediction: {round(Y_test[0],3)}')